In [ ]:
# this code is to perform the sequence clustering, to use this code
#1. prepare nextcalde clade info and aligned fasta file, can use nextclade CLI to do this easily
#2. install necessary package

In [4]:
import pandas as pd

# Step 1: Load the metadata file
metadata_file = "output_04_14/nextclade_04_14.tsv"
df = pd.read_csv(metadata_file, sep ="\t")

/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_84775/4274116231.py:5: DtypeWarning: Columns (28,50,51,52,53,54,55,56,57,58,59,61,75,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_file, sep ="\t")


In [ ]:
# Step 2: Drop lineages that occur only once, please remember to add this number for your signletons cluster
original_count = len(df)
df_filtered = df[df.groupby("Nextclade_pango")["Nextclade_pango"].transform("count") > 1]
filtered_count = len(df_filtered)
dropped_count = original_count - filtered_count
print(f"Number of sequences dropped (unique lineages): {dropped_count}")

# Step 3: Group by Lineage and extract sequence strain
lineage_to_seqnames = (
    df_filtered.groupby("Nextclade_pango")["seqName"]
    .apply(list)
    .to_dict()
)

Number of sequences dropped (unique lineages): 293


In [ ]:
# Step 4: Group sequenes from the aligned fasta file to individual file, I highly recommend using this approach as it is much easiler to run small file each time compare to super large file
from Bio import SeqIO
import os

input_fasta = "04_14_nextclade.aligned.fasta"  # replace with actual path

# Parse once, and store records by name for fast lookup
all_records = SeqIO.to_dict(SeqIO.parse(input_fasta, "fasta"))

# Make sure the output directory exists
os.makedirs("lineage_group_fasta", exist_ok=True)

# Then, write out grouped FASTAs
for lineage, names in lineage_to_seqnames.items():
    output_file = f"lineage_group_fasta/{lineage}_group.fasta"
    
    # Identify missing names
    missing_names = [name for name in names if name not in all_records]
    if missing_names:
        print(f"Warning: {len(missing_names)} sequences not found for lineage {lineage}")
        # Optionally, print the missing sequence names
        # print(f"Missing: {missing_names}")

    # Write only found sequences
    selected_records = [all_records[name] for name in names if name in all_records]
    
    if selected_records:
        with open(output_file, "w") as out_f:
            SeqIO.write(selected_records, out_f, "fasta")
    else:
        print(f"Warning: No sequences written for lineage {lineage} — all sequence names missing.")


In [ ]:
# Now, use pairsnp to caculate the distance matric of a given clade
import os
import numpy as np
import pandas as pd
from pairsnp import calculate_snp_matrix, calculate_distance_matrix

base_dir = "MN_data/lineage_group_fasta/"
test_one = True  # Set True to test just one group

# Track which clades are empty
empty_clades = []

# Loop through each grouped FASTA file
for filename in sorted(os.listdir(base_dir)):
    if filename.endswith("_group.fasta"):
        fasta_path = os.path.join(base_dir, filename)
        lineage_name = filename.replace("_group.fasta", "")

        try:
            print(f"Processing: {fasta_path}")

            # Run PairSNP analysis
            sparse_matrix, consensus, seq_names = calculate_snp_matrix(fasta_path)
            d = calculate_distance_matrix(sparse_matrix, consensus, "dist", False)

            # Save labeled distance matrix
            df = pd.DataFrame(d, index=seq_names, columns=seq_names)
            output_csv = os.path.join(base_dir, f"distance_matrix_group/{lineage_name}_snp_distance_matrix.csv")
            df.to_csv(output_csv)

        except ValueError as e:
            if "No sequences found" in str(e):
                print(f"⚠️ Skipping {lineage_name} — no sequences found.")
                empty_clades.append(lineage_name)
            else:
                raise  # Re-raise other unexpected errors

# Save the list of empty clades
if empty_clades:
    log_path = os.path.join(base_dir, "empty_clades_log.txt")
    with open(log_path, "w") as log_file:
        for clade in empty_clades:
            log_file.write(f"{clade}\n")

    print(f"\n📝 Empty clades saved to: {log_path}")


In [ ]:
#after caculate the distance matrix, now lets find the identical sequence, the code here is for identicall sequence, you can change the distance to include more seq
# the code is validated on 04-15 fix the duplicated problem
#the code will generate two file, one include the cluster summary, one include the indiviudal seq within a given cluster
import os
import pandas as pd
from collections import defaultdict
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import numpy as np

# === Set input directory containing *_snp_distance_matrix.csv files ===
input_dir = "MN_data/lineage_group_fasta/distance_matrix_group/"

# === Containers for outputs ===
cluster_summary = []
detailed_clusters = []

def find_clusters(distance_df):
    """Return list of clusters (sets) using graph connectivity (distance = 0)"""
    matrix = (distance_df.values == 0).astype(int)
    np.fill_diagonal(matrix, 0)  # Ignore self-distances
    graph = csr_matrix(matrix)

    # Compute connected components
    n_components, labels = connected_components(csgraph=graph, directed=False)
    clusters = defaultdict(set)

    for seq, label in zip(distance_df.index, labels):
        clusters[label].add(seq)

    return list(clusters.values())


# === Loop through files ===
for file in sorted(os.listdir(input_dir)):
    if not file.endswith("_snp_distance_matrix.csv"):
        continue

    file_path = os.path.join(input_dir, file)
    lineage = file.replace("_snp_distance_matrix.csv", "")

    try:
        df = pd.read_csv(file_path, index_col=0)

        # Find clusters of identical sequences
        clusters = find_clusters(df)

        # Count clusters by size
        size_count = defaultdict(int)
        for cluster in clusters:
            size_count[len(cluster)] += 1

        # Add summary
        for size, count in size_count.items():
            cluster_summary.append({
                "Lineage": lineage,
                "Cluster_Size": size,
                "Count": count
            })

        # Add detailed members
        for i, cluster in enumerate(clusters, 1):
            detailed_clusters.append({
                "Lineage": lineage,
                "Cluster_ID": f"{lineage}_Cluster_{i}",
                "Members": ";".join(sorted(cluster))
            })

    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

# === Convert to DataFrames ===
summary_df = pd.DataFrame(cluster_summary)
detailed_df = pd.DataFrame(detailed_clusters)

# === Save results ===
summary_df.to_csv(os.path.join(input_dir, "all_lineage_cluster_summary.csv"), index=False)
detailed_df.to_csv(os.path.join(input_dir, "all_lineage_cluster_membership.csv"), index=False)

print("✅ Finished. Summary and detailed cluster membership saved.")


In [ ]:
#additonal code for count number of each cluster
import pandas as pd

# === Load the cluster summary file ===
summary_path = "MN_data/lineage_group_fasta/distance_matrix_group/all_lineage_cluster_summary.csv"  # Update if needed
summary_df = pd.read_csv(summary_path)

# === Group by Cluster_Size and sum the counts ===
size_summary = summary_df.groupby("Cluster_Size")["Count"].sum().reset_index()

# === Save detailed counts for all cluster sizes ===
size_summary.to_csv("MN_data/lineage_group_fasta/distance_matrix_group/cluster_size_summary_04_15.csv", index=False)

print("✅ Saved detailed cluster size summary to 'cluster_size_summary.csv'")



In [ ]:
#code to extract metadata to cluster from the large metadata file
# the code is slighlty redunant as the the seqName contains strain name from different source,but the point is to extract all the associated metadata of a given cluster for RR analysis
import pandas as pd
import os
import re

# === Load Data ===
membership_df = pd.read_csv("MN_data/lineage_group_fasta/distance_matrix_group/all_lineage_cluster_membership.csv")
metadata_df = pd.read_csv("MN_data/matched_metadata_final_drop_doplicate.csv")

# === Output folder ===
output_dir = "cluster_metadata_outputs"
os.makedirs(output_dir, exist_ok=True)

# === Helper function ===
def normalize_member_ids(members_raw):
    members_fixed = []
    for m in members_raw:
        match = re.search(r'hCoV-19/USA/([A-Z]{2}-[^/]+)', m)
        if match:
            extracted = match.group(1)
            print(f"✅ Match: {m} → {extracted}")
            members_fixed.append(extracted)
        else:
            print(f"❌ Failed match: {m}")
            members_fixed.append(None)
    return members_fixed

# === Skip clusters with only 1 member ===
membership_df["Member_Count"] = membership_df["Members"].apply(lambda x: len(x.split(";")))

for idx, row in membership_df.iterrows():
    cluster_id = row["Cluster_ID"]
    members_raw = row["Members"].split(";")

    if len(members_raw) <= 1:
        continue  # ✅ skip singleton clusters

    # Normalize member names
    members_fixed = normalize_member_ids(members_raw)

    if None in members_fixed:
        raise ValueError(f"❌ Cluster '{cluster_id}' has unrecognized member format: {members_raw}")

    # Match both dash and underscore forms
    members_fixed_all = set(members_fixed + [m.replace("-", "_") for m in members_fixed if m])
    metadata_matches = metadata_df[metadata_df["normalized_strain"].isin(members_fixed_all)].copy()
    metadata_matches["Cluster_ID"] = cluster_id

    # Raise error if any are missing
    matched_set = set(metadata_matches["normalized_strain"])
    missing = [m for m in members_fixed if m and m not in matched_set and m.replace("-", "_") not in matched_set]
    if missing:
        raise ValueError(f"❌ Cluster '{cluster_id}' has missing members: {missing}")

    # Save metadata for this cluster
    output_path = os.path.join(output_dir, f"{cluster_id}_metadata.tsv")
    metadata_matches.to_csv(output_path, sep='\t', index=False)
    print(f"✅ Saved: {cluster_id} ({len(metadata_matches)} records)")


In [24]:
#code for only keeping metadata row in the fastafile
import pandas as pd

# Load input files
metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")

# Step 1: Extract core ID from nextclade seqName (e.g., hCoV-19/USA/MN-CDC-XXX/2021)
nextclade_df['core_id'] = nextclade_df['seqName'].str.extract(r'hCoV-19/USA/([^/]+)', expand=False)

# Step 2: Replace underscores with dashes for consistency
nextclade_df['core_id'] = nextclade_df['core_id'].str.replace('_', '-', regex=False)
metadata_df['normalized_strain'] = metadata_df['strain'].str.replace('_', '-', regex=False)

# Step 3: Filter metadata to keep only matched strains
matched_metadata_df = metadata_df[metadata_df['normalized_strain'].isin(nextclade_df['core_id'])]

# Step 4: Save the matched metadata to a new file
matched_metadata_df.to_csv("matched_metadata_final.tsv", sep="\t", index=False)

# Step 5: Report number of matches
print(f"Number of matched metadata rows: {matched_metadata_df.shape[0]}")


/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/4142577797.py:5: DtypeWarning: Columns (24,25,27,28,30,31,32,33,35,36,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/4142577797.py:6: DtypeWarning: Columns (28,50,51,52,53,54,55,56,57,58,59,61,75,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")


Number of matched metadata rows: 82947


In [23]:
#checking for which row is present in the metadata

# Load metadata and Nextclade files
metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")

# Extract core ID from both files (e.g., MN-MDH-32441 or MN_UMGC_28146)
nextclade_df['core_id'] = nextclade_df['seqName'].str.extract(r'(MN[-_][A-Z]+[-_]\d+)')
metadata_df['core_id'] = metadata_df['covid_virus_name'].str.extract(r'(MN[-_][A-Z]+[-_]\d+)')

# Normalize both core_id columns by replacing underscores with dashes
nextclade_df['core_id'] = nextclade_df['core_id'].str.replace('_', '-', regex=False)
metadata_df['core_id'] = metadata_df['core_id'].str.replace('_', '-', regex=False)

# Filter to matched entries
matched_df = nextclade_df[nextclade_df['core_id'].isin(metadata_df['core_id'])]

# Print total number of matched entries
print("Number of matched sequences:", matched_df.shape[0])



/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/3735541180.py:4: DtypeWarning: Columns (24,25,27,28,30,31,32,33,35,36,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/3735541180.py:5: DtypeWarning: Columns (28,50,51,52,53,54,55,56,57,58,59,61,75,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")


Number of matched sequences: 82946
